In [ ]:
from annealed_flow_transport.train import run_experiment
import numpy as np
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
from annealed_flow_transport.many_well_plotting import plot, plot_marginal_pair
from annealed_flow_transport.densities import ManyWell
from annealed_flow_transport.resampling import log_effective_sample_size

import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 100

In [ ]:
from configs.many_well_original import get_config as many_well_original_get_config
from configs.many_well import get_config as many_well_get_config

In [ ]:
def get_experiment_config(seed, experiment_type = "original", long=True):
    # Set long to False to run just a short version of the experiment.
    if experiment_type == "original":
        exp_config = many_well_original_get_config()
        exp_config.n_samples_plotting = 100
    else:
        assert experiment_type == "custom"
        exp_config = many_well_get_config()
    if not long: # Run for short
        exp_config.craft_num_iters = 200
        exp_config.report_step = int(exp_config.craft_num_iters // 6)
    exp_config.seed = seed
    exp_config.params_filename = f"checkpoint_blob_{experiment_type}_L{long}_seed{exp_config.seed}"
    
    n_mill_target_eval = exp_config.craft_batch_size*exp_config.craft_num_iters*exp_config.num_temps*exp_config.mcmc_config.hmc_num_leapfrog_steps/1e6
    print(f"experiment using {n_mill_target_eval} target evaluations")
    return exp_config

# Train using the config from the original CRAFT repo

In [ ]:
exp_config = get_experiment_config(0, "original")
results0 = run_experiment(exp_config)

In [ ]:
exp_config = get_experiment_config(1, "original")
results1 = run_experiment(exp_config)

In [ ]:
exp_config = get_experiment_config(2, "original")
results2 = run_experiment(exp_config)

# Train using config similar to FAB

In [ ]:
exp_config = get_experiment_config(0, "custom")
results0 = run_experiment(exp_config)

In [ ]:
exp_config = get_experiment_config(1, "custom")
results1 = run_experiment(exp_config)

In [ ]:
exp_config = get_experiment_config(2, "custom")
results2 = run_experiment(exp_config)

# Evaluation
Load checkpoints and evaluate

In [ ]:
from evaluation import evaluate_many_well, load_checkpoint, make_forward_pass_func, \
    make_get_ess, make_get_resample_info, get_flow_init_params
import scipy

In [ ]:
target_density = ManyWell(exp_config.final_config.density, (32,))

In [ ]:
eval_batch_size = 1000

In [ ]:
def evaluate_Z_estimation(seed, experiment_type = "original", long=True, 
                          eval_batch_size=1000, n_eval=50, flow_identity=False):
    exp_config = get_experiment_config(seed, experiment_type, long)
    if flow_identity:
        transition_params = get_flow_init_params(exp_config)
    else:
        transition_params = load_checkpoint(exp_config.params_filename)
    forward_pass_function = make_forward_pass_func(exp_config, transition_params=transition_params,
                                                  eval_batch_size=eval_batch_size)
    key = jax.random.PRNGKey(0)
    abs_errors = []
    for i in range(n_eval):
        key, subkey = jax.random.split(key)
        particle_state = forward_pass_function(subkey)
        log_Z_estimate = particle_state.log_normalizer_estimate
        relative_error = jnp.exp(log_Z_estimate - target_density.log_Z) - 1
        abs_error = jnp.abs(relative_error)
        abs_errors.append(abs_error)
    return jnp.stack(abs_errors)

#### Original

In [ ]:
# standard
abs_errors0 = evaluate_Z_estimation(0, "original")
jnp.mean(abs_errors0)

In [ ]:
# standard
abs_errors1 = evaluate_Z_estimation(1, "original")
jnp.mean(abs_errors1)

In [ ]:
abs_errors2 = evaluate_Z_estimation(2, "original")
jnp.mean(abs_errors2)

In [ ]:
# for table
vals = [float(jnp.mean(abs_errors0)), float(jnp.mean(abs_errors1)), jnp.mean(abs_errors2)]
sum(vals) / 3, scipy.stats.sem(vals, ddof=0) 

#### Similar to FAB

In [ ]:
abs_errors0 = evaluate_Z_estimation(0, "custom")
jnp.mean(abs_errors0)

In [ ]:
abs_errors1 = evaluate_Z_estimation(1, "custom")
jnp.mean(abs_errors1)

In [ ]:
abs_errors2 = evaluate_Z_estimation(2, "custom")
jnp.mean(abs_errors2)

In [ ]:
# for table
vals = [float(jnp.mean(abs_errors0)), float(jnp.mean(abs_errors1)), jnp.mean(abs_errors2)]
sum(vals) / 3, scipy.stats.sem(vals, ddof=0) 